# Azure Language - Named Entity Recognition

This notebook demonstrates how to use the Azure Language Analyze Text API to perform
Named Entity Recognition (NER) on a business contract between Aperture Laboratories
and Umbrella Corporation.

## Prerequisites

Set the following environment variables:
- `FOUNDRY_ENDPOINT` - Azure Language service endpoint
- `FOUNDRY_API_KEY` - API key for authentication

In [ ]:
import json
import os

import requests
from rich.console import Console
from rich.panel import Panel
from rich.table import Table

## Load Sample Data

Load the business contract from the sample data file.

In [ ]:
with open(
    "02-NamedEntityRecognition-data/business_contract.txt", encoding="utf-8"
) as f:
    contract_text = f.read()

print(f"Loaded contract: {len(contract_text)} characters")
print(f"\nFirst 500 characters:\n{contract_text[:500]}...")

## API Setup

Configure the Azure Language API endpoint and authentication.

In [ ]:
endpoint = os.environ["FOUNDRY_ENDPOINT"]
api_key = os.environ["FOUNDRY_API_KEY"]

# Construct the Analyze Text API URL
api_url = f"{endpoint}/language/:analyze-text?api-version=2024-11-01"

headers = {
    "Ocp-Apim-Subscription-Key": api_key,
    "Content-Type": "application/json",
}

print(f"API Endpoint: {endpoint}")
print("API Key: [CONFIGURED]")

## Make API Call

Send the contract text to the Named Entity Recognition API.

In [ ]:
# Prepare the document for the API
documents = [{"id": "1", "language": "en", "text": contract_text}]

# Build the request payload
payload = {
    "kind": "EntityRecognition",
    "parameters": {"modelVersion": "latest"},
    "analysisInput": {"documents": documents},
}

# Make the API request
response = requests.post(api_url, headers=headers, json=payload, timeout=30)
response.raise_for_status()
result = response.json()

print("API call successful!")

## Raw API Results

Display the complete JSON response from the API.

In [ ]:
print(json.dumps(result, indent=2))

## Human-Friendly Results

Present the named entities in a readable format, organized by category.

In [ ]:
console = Console()

# Extract entities from the first document
entities = result["results"]["documents"][0]["entities"]

# Create main entity table
table = Table(title="Named Entities Recognized in Contract")
table.add_column("Entity", style="cyan")
table.add_column("Category", style="green")
table.add_column("Subcategory", style="dim")
table.add_column("Confidence", justify="center")

for entity in entities:
    confidence = f"{entity['confidenceScore']:.0%}"
    subcategory = entity.get("subcategory", "-")
    table.add_row(entity["text"], entity["category"], subcategory, confidence)

console.print(table)

# Group entities by category for summary
categories: dict[str, list[str]] = {}
for entity in entities:
    cat = entity["category"]
    if cat not in categories:
        categories[cat] = []
    if entity["text"] not in categories[cat]:
        categories[cat].append(entity["text"])

# Print category summary
console.print("\n")
summary_table = Table(title="Entity Summary by Category")
summary_table.add_column("Category", style="bold green")
summary_table.add_column("Count", justify="center", style="cyan")
summary_table.add_column("Unique Entities", style="dim")

for cat, ents in sorted(categories.items()):
    entities_preview = ", ".join(ents[:5])
    if len(ents) > 5:
        entities_preview += f" (+{len(ents) - 5} more)"
    summary_table.add_row(cat, str(len(ents)), entities_preview)

console.print(summary_table)

# Key contract parties
orgs = categories.get("Organization", [])
people = categories.get("Person", [])
locations = categories.get("Location", []) + categories.get("GPE", [])

key_info = f"""
[bold]Key Contract Information Extracted:[/bold]

[cyan]Organizations:[/cyan] {", ".join(orgs[:5]) if orgs else "None found"}
[cyan]People:[/cyan] {", ".join(people[:5]) if people else "None found"}
[cyan]Locations:[/cyan] {", ".join(locations[:5]) if locations else "None found"}
"""

console.print(Panel(key_info, title="Contract Parties & Locations"))